# Tutorial of out-of-sample high-resolution prediction

This tutorial demonstrates out-of-sample high-resolution gene expression prediction using the Xenium lung adenocarcinoma dataset.
The evaluation is performed on slice E1, while the model is trained on a slice from an independent patient.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import scanpy as sc
from MicroMap import MicroMap

## Download data from Google Drive

The raw data used here are identical to those in the in-sample tutorial.
If you have already downloaded the dataset in the previous tutorial, this step can be skipped.

In [2]:
import os

# Folder link
GDRIVE_URL = "https://drive.google.com/drive/folders/1jCCmvksz9r1DC-W-2mpXzNqpU9c4Ab8R"
DATA_DIR = "data"

# Create data directory if not exists
os.makedirs(DATA_DIR, exist_ok=True)

# Install gdown if needed
try:
    import gdown
except ImportError:
    !pip install -q gdown
    import gdown

# Download entire folder
gdown.download_folder(GDRIVE_URL, output=DATA_DIR, quiet=False)

print("\n Data download finished!")
print("Files in data/:", os.listdir(DATA_DIR))


Retrieving folder contents


Processing file 195ERY62DFTQHhjlTaex9hWhlcxmPkysE adata_gt_E1.h5ad
Processing file 1Gaem6GsJfNGo7nffEwRUa2k0Xg6HtlL_ adata_visium_cross_sample.h5ad
Processing file 1wX2jJPV4PjSveY3zjP6328TNzpY9KmIf adata_visium_E1.h5ad
Processing file 1y0rxIoiHMaAO7WwDgixjY9UjZHBM_4xD HE_cross_sample.png
Processing file 1LFOSWH5xWUuBTd89Uwt-lIS3ao4dv8Uz HE_E1.png


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=195ERY62DFTQHhjlTaex9hWhlcxmPkysE
From (redirected): https://drive.google.com/uc?id=195ERY62DFTQHhjlTaex9hWhlcxmPkysE&confirm=t&uuid=e57a63c5-d91b-4a53-93ad-fd27fe7b315c
To: /data/yyyu/test/imint/tutorial/data/adata_gt_E1.h5ad
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 266M/266M [00:28<00:00, 9.47MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Gaem6GsJfNGo7nffEwRUa2k0Xg6HtlL_
To: /data/yyyu/test/imint/tutorial/data/adata_visium_cross_sample.h5ad
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.79M/1.79M [00:01<00:00, 982kB/s]
Downloading...
From: https://d


 Data download finished!
Files in data/: ['adata_gt_rep1.h5ad', 'HE_cross_sample.png0qeco020.part', 'adata_gt_E1.h5ada1_q43wu.part', 'HE_rep2.png', 'adata_visium_E2.h5ad', 'adata_gt_E1.h5ad', 'adata_visium_rep2.h5ad', 'adata_visium_cross_sample.h5ad', 'adata_visium_E1.h5ad', 'HE_E1.png', 'HE_cross_sample.png']


Download completed


## Load low resolution data

Next, we load the low-resolution synthetic gene expression data, which have high-resolution ground-truth.

In [3]:
adata = sc.read('./data/adata_visium_cross_sample.h5ad')
adata

adata1 = sc.read('./data/adata_visium_E1.h5ad')
adata1

genes = sorted(adata.var_names.intersection(adata1.var_names))
adata = adata[:,genes].copy()
adata

AnnData object with n_obs × n_vars = 1112 × 156
    obs: 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'n_counts'
    var: 'n_cells'
    obsm: 'spatial'

## Construct and run with MicroMap object

We then initialize a MicroMap object and compute spot-specific latent priors.

In [4]:
# construct 
micro =  MicroMap( device = 'cuda:0', out_path='out_of_sample')

# calculate expression latent distrobution prior for each spot
micro.cal_spot_prior(adata.X)

Preprocessing spot level gene expression with 1112 spots and 156 genes ... 
Calculating spot specific prior ... 


  0%|▊                                                                                                                                                                          | 1/200 [00:01<05:20,  1.61s/it]

[Pretrain Epoch 0] total loss = 26.1388


  1%|█▋                                                                                                                                                                         | 2/200 [00:02<03:16,  1.01it/s]

[Pretrain Epoch 1] total loss = 25.2004


...

[Pretrain Epoch 197] total loss = 14.9431


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 199/200 [01:58<00:00,  1.73it/s]

[Pretrain Epoch 198] total loss = 14.8901


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:59<00:00,  1.67it/s]

[Pretrain Epoch 199] total loss = 14.8712


Image features are extracted using the pathology foundation model UNI, which may take approximately 20 minutes depending on hardware and system configuration. Users are required to download the pretrained model weights pytorch_model.bin in advance from:
https://github.com/mahmoodlab/UNI

In [5]:
# calculate image features via foundation model UNI (https://github.com/mahmoodlab/UNI)
micro.cal_token_feats(   img_path = './data/HE_cross_sample.png',
                         UNI_path = '/data/yyyu/test/UNI/code_raw/assets/ckpts/vit_large_patch16_224.dinov2.uni_mass100k/pytorch_model.bin',
                         scale = 0.4
                         )

Creating and loading model ... 
Extracting features... 
Saving features ...
Saved to: out_of_sample/features.pt


Subsequently, the main model is trained to predict high-resolution gene expression.

In [6]:
# train the backbone model
micro.train( 
             spot_coord = adata.obsm['spatial'][:,[1,0]], 
             spot_radius = 137.5,
             spot_count = adata.X,
             genes = adata.var_names.tolist(),
          )

  0%|▌                                                                                                                                                                          | 1/300 [00:01<09:08,  1.84s/it]

Epoch 1: total=12.9875, nb=6.3456, size=1.2592, smooth=3.2949, cos_gene=2.3624, cos_cell=2.0020, kl_token=3.3270, 


  1%|█▏                                                                                                                                                                         | 2/300 [00:03<08:58,  1.81s/it]

Epoch 2: total=5.6518, nb=2.4495, size=0.2585, smooth=1.0812, cos_gene=1.2928, cos_cell=0.8549, kl_token=3.0005, 


  1%|█▋                                                                                                                                                                         | 3/300 [00:05<08:41,  1.76s/it]

...

Epoch 298: total=2.7116, nb=1.6054, size=0.0024, smooth=0.3214, cos_gene=0.2275, cos_cell=0.1032, kl_token=1.9340, 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 299/300 [09:30<00:01,  1.93s/it]

Epoch 299: total=2.7080, nb=1.6051, size=0.0018, smooth=0.3222, cos_gene=0.2295, cos_cell=0.1034, kl_token=1.9569, 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [09:32<00:00,  1.91s/it]

Epoch 300: total=2.7108, nb=1.6066, size=0.0027, smooth=0.3230, cos_gene=0.2261, cos_cell=0.1040, kl_token=1.9291, 


In [7]:
# prediction
import torch

micro.load_token_feats( feats_file = 'in_sample/features.pt', scale = 0.4)

adata_hir = micro.predict(  genes = adata.var_names.tolist(), extract_mask=False)
adata_hir # High-resolution gene expression across the entire H&E image

(687960, 156)


AnnData object with n_obs × n_vars = 687960 × 156
    obs: 'patch_y', 'patch_x'
    obsm: 'spatial', 'latent'

## Evaluation

Finally, we load the 8 μm ground-truth data and evaluate the prediction performance using multiple quantitative metrics, including Pearson correlation coefficient (PCC), Spearman rank correlation coefficient (RCC), root mean squared error (RMSE), and structural similarity index measure (SSIM).

In [8]:
from MicroMap.evaluate import evaluation_func, normalize_count

In [9]:
adata_gt = sc.read('./data/adata_gt_E1.h5ad')
adata_gt = adata_gt[:,genes].copy()
adata_gt

AnnData object with n_obs × n_vars = 212081 × 156
    obs: 'patch_y', 'patch_x', 'n_counts'
    var: 'moranI'
    obsm: 'patch', 'spatial'

In [10]:
genes = adata_gt.var_names
tokens = adata_gt.obs_names
adata_use = adata_hir[tokens, genes].copy()
adata_use

expr_gt = adata_gt.to_df()
expr_gt_norm = normalize_count(expr_gt)

expr_use = adata_use.to_df()
expr_use_norm = normalize_count(expr_use)

In [11]:
patch_shape = (adata_gt.obs[['patch_x', 'patch_y']].max()[0]+1, adata_gt.obs[['patch_x', 'patch_y']].max()[1]+1)
index_gt = adata_gt.obs[['patch_x', 'patch_y']].values
index_gt = (index_gt[:,0], index_gt[:,1])

metric = evaluation_func(expr_use_norm, expr_gt_norm, genes,
                         do_spearman=True, spearman_subsample=100000, spearman_jobs=8,
                         do_ssim=True, ssim_kwargs={'index_gt':index_gt, 'patch_shape':patch_shape})

moranI = adata_gt.var['moranI']
metric[moranI>0.2].median()

PCC     0.343896
RMSE    0.058676
RCC     0.251338
SSIM    0.707103
dtype: float32